In [ ]:
! python -m spacy download ru_core_news_md
! pip install catboost
! pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 24.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.2 MB/s eta 0:00:00


In [ ]:
import re
import ast
import nltk
import json
import spacy
import gensim

import pandas as pd
import numpy as np


from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
with open("/content/news.txt", "r") as f:
   lines_list = f.readlines()

In [ ]:
lines_list = list(map(lambda x: x.split("\t"), lines_list))

In [ ]:
df = pd.DataFrame(lines_list, columns = ['title',"summary_news","text_news"])

In [ ]:
df

,title,summary_news,text_news
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс..."
...,...,...,...
9995,economics,Бывший пропагандист Буша нашел работу в торгов...,Бывший советник президента США Джорджа Буша по...
9996,travel,Россияне предпочли смартфон любимому человеку ...,Портал онлайн-бронирования отелей Hotels.com о...
9997,life,Потерявшаяся пять недель назад американка найд...,"В США два туриста обнаружили в лесу женщину, п..."
9998,media,"НТВ покажет решающие матчи ""Зенита"" и ЦСКА в Л...",Телеканал НТВ покажет в прямом эфире решающие ...


In [ ]:
nlp = spacy.load("ru_core_news_md")

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^\wа-яё]+", " ", text)
    doc = nlp(text)

    return [token.lemma_ for token in doc]

def preprocessing_text(text):
    text = tokenize(text)

    stop_words = set(stopwords.words('russian'))

    filtered_tokens = [word for word in text if word not in stop_words and word.isalpha()]

    return filtered_tokens


In [ ]:
# df['preprocess_text'] = df['text_news'].apply(preprocessing_text) Датасет уже готов, загружаем из окружения

df = pd.read_csv("/content/dataset_final.csv", index_col=0)

In [ ]:
df

,title,summary_news,text_news,preprocess_text
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...,"['парусный', 'гонка', 'giraglia', 'rolex', 'cu..."
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...,"['шведский', 'хоккеист', 'матс', 'сундин', 'на..."
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г...","['гран', 'конкурса', 'брэнд', 'год', 'effie', ..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...,"['цена', 'американский', 'нефть', 'wti', 'лонд..."
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс...","['сбербанк', 'выставить', 'продажа', 'долг', '..."
...,...,...,...,...
9995,economics,Бывший пропагандист Буша нашел работу в торгов...,Бывший советник президента США Джорджа Буша по...,"['бывший', 'советник', 'президент', 'сша', 'дж..."
9996,travel,Россияне предпочли смартфон любимому человеку ...,Портал онлайн-бронирования отелей Hotels.com о...,"['портал', 'онлайн', 'бронирование', 'отель', ..."
9997,life,Потерявшаяся пять недель назад американка найд...,"В США два туриста обнаружили в лесу женщину, п...","['сша', 'турист', 'обнаружить', 'лес', 'женщин..."
9998,media,"НТВ покажет решающие матчи ""Зенита"" и ЦСКА в Л...",Телеканал НТВ покажет в прямом эфире решающие ...,"['телеканал', 'нтв', 'показать', 'прямой', 'эф..."


In [ ]:
# df.to_csv("dataset_final.csv") # датасет уже сохранили

X_train, X_test, y_train, y_test = train_test_split(
    df['preprocess_text'], df['title'], test_size=0.33, random_state=42)

In [ ]:
X_train = pd.DataFrame(X_train)

X_test = pd.DataFrame(X_test)

In [ ]:
# model = Word2Vec(sentences = X_train,
#                  vector_size=100, window=5,
#                  min_count=3, workers=4,
#                  sg=1, epochs=10
#                  )


# model.save("/content/drive/MyDrive/word2vec.model") модель уже обучения, загружаем из окружения

model_word2vec = Word2Vec.load("/content/drive/MyDrive/word2vec.model")


In [ ]:
def word2vec_fun_ver1(list_of_word):

  i = 0

  list_of_word = ast.literal_eval(list_of_word)

  word_average = np.zeros((100,))

  for word in list_of_word:
      try:
          word_average += model_word2vec.wv[word].copy()
          i += 1
      except Exception as e:
          pass

  word_average = word_average / i

  return word_average


def word2vec_fun_ver2(list_of_word): # вместо усреднения все элементы масштабируем: вычитаем среднее и делим на стандартное отклонение

  i = 0

  list_of_word = ast.literal_eval(list_of_word)

  word_average = np.zeros((100,))

  for word in list_of_word:
      try:
          word_average += model_word2vec.wv[word].copy()
          i += 1
      except Exception as e:
          pass

  mean = np.mean(word_average, axis=0)      # среднее по каждому признаку
  std = np.std(word_average, axis=0)

  word_average = (word_average - mean) / std

  return word_average

In [ ]:
X_train['vector'] = X_train['preprocess_text'].apply(word2vec_fun_ver1)

X_test['vector'] = X_test['preprocess_text'].apply(word2vec_fun_ver1)


In [ ]:
X_train_vectors = np.vstack(X_train['vector'].values)
X_test_vectors = np.vstack(X_test['vector'].values)


iterations = 30
learning_rate = 1
depth = 4

model = CatBoostClassifier(iterations = iterations,
                           learning_rate = learning_rate,
                           depth = depth)
# Fit model
model.fit(X_train_vectors, y_train)

0:	learn: 1.7784966	total: 91.9ms	remaining: 2.66s
1:	learn: 1.6168983	total: 163ms	remaining: 2.29s
2:	learn: 1.3613041	total: 238ms	remaining: 2.14s
3:	learn: 1.1775590	total: 310ms	remaining: 2.01s
4:	learn: 1.0787063	total: 379ms	remaining: 1.9s
5:	learn: 0.9853916	total: 447ms	remaining: 1.79s
6:	learn: 0.9094035	total: 522ms	remaining: 1.72s
7:	learn: 0.8561820	total: 590ms	remaining: 1.62s
8:	learn: 0.8343124	total: 668ms	remaining: 1.56s
9:	learn: 0.8043592	total: 747ms	remaining: 1.49s
10:	learn: 0.7685388	total: 819ms	remaining: 1.42s
11:	learn: 0.7548186	total: 885ms	remaining: 1.33s
12:	learn: 0.7496342	total: 949ms	remaining: 1.24s
13:	learn: 0.7309802	total: 1.02s	remaining: 1.17s
14:	learn: 0.7127767	total: 1.09s	remaining: 1.09s
15:	learn: 0.7015975	total: 1.15s	remaining: 1.01s
16:	learn: 0.6901249	total: 1.22s	remaining: 930ms
17:	learn: 0.6733073	total: 1.28s	remaining: 856ms
18:	learn: 0.6625550	total: 1.35s	remaining: 782ms
19:	learn: 0.6364489	total: 1.42s	remaini

In [ ]:
f1_score(y_test, model.predict(X_test_vectors), average = "weighted")

0.7686346351589202

In [ ]:
X_train['vector_2'] = X_train['preprocess_text'].apply(word2vec_fun_ver2)

X_test['vector_2'] = X_test['preprocess_text'].apply(word2vec_fun_ver2)

X_train_vectors_2 = np.vstack(X_train['vector_2'].values)
X_test_vectors_2 = np.vstack(X_test['vector_2'].values)

model = CatBoostClassifier(iterations = iterations,
                           learning_rate = learning_rate,
                           depth = depth)
# Fit model
model.fit(X_train_vectors_2, y_train)


0:	learn: 1.7264293	total: 93.8ms	remaining: 2.72s
1:	learn: 1.8964777	total: 169ms	remaining: 2.37s
2:	learn: 1.9892970	total: 243ms	remaining: 2.19s
3:	learn: 1.8460003	total: 310ms	remaining: 2.02s
4:	learn: 1.6639446	total: 373ms	remaining: 1.87s
5:	learn: 1.4987560	total: 441ms	remaining: 1.76s
6:	learn: 1.5476120	total: 510ms	remaining: 1.68s
7:	learn: 1.3328219	total: 594ms	remaining: 1.63s
8:	learn: 1.2395040	total: 668ms	remaining: 1.56s
9:	learn: 1.2139732	total: 729ms	remaining: 1.46s
10:	learn: 1.1497721	total: 793ms	remaining: 1.37s
11:	learn: 1.1124500	total: 856ms	remaining: 1.28s
12:	learn: 1.0561227	total: 924ms	remaining: 1.21s
13:	learn: 1.0127392	total: 997ms	remaining: 1.14s
14:	learn: 0.9807038	total: 1.06s	remaining: 1.06s
15:	learn: 0.9587533	total: 1.13s	remaining: 991ms
16:	learn: 0.9382259	total: 1.2s	remaining: 916ms
17:	learn: 0.9200257	total: 1.26s	remaining: 842ms
18:	learn: 0.8791484	total: 1.33s	remaining: 771ms
19:	learn: 0.8675614	total: 1.4s	remainin

In [ ]:
f1_score(y_test, model.predict(X_test_vectors_2), average = "weighted")

0.7547159948119525

In [166]:
model_word2vec.wv.most_similar("политика", topn=5)


[('монетарный', 0.7256817817687988),
 ('общественно', 0.701324999332428),
 ('высокорискованную', 0.6972029805183411),
 ('редакционный', 0.6871280074119568),
 ('вмешиваться', 0.6807732582092285)]